In [1]:
import os
import pandas as pd
from tkinter import Tk, Label, OptionMenu, StringVar, Button, Canvas, Scrollbar, Frame
from PIL import Image, ImageTk, ImageOps

class ImageLabeler:
    """
    A class to load images from a folder, allow classification through a GUI,
    and save the classifications to an Excel file.

    Attributes:
    - folder_path (str): The path to the folder containing images.
    - sample_name (str): The base name for the Excel file (without extension).
    - classification_options (list): Custom classification labels to display in the dropdown, 'Delete' is always an option.
    - images (list): List of image filenames in the folder.

    Methods:
    - load_images: Loads and sorts images in the specified folder.
    - setup_gui: Sets up the GUI to display images with classification options.
    - save_to_excel: Saves the classifications to an Excel file with '_labeled.xlsx' appended to the sample name.
    """

    def __init__(self, folder_path, classification_options, sample_name):
        """
        Initializes the ImageLabeler class with the folder path, classification options, and sample name.

        Parameters:
        - folder_path (str): The path to the folder containing images.
        - classification_options (list): A list of classification labels to choose from.
        - sample_name (str): The base name for the output Excel file.
        """
        self.folder_path = folder_path
        self.sample_name = sample_name
        self.classification_options = ["Delete"] + classification_options
        self.images = self.load_images()

    def load_images(self):
        """
        Loads and sorts image files in the specified folder.

        Returns:
        - list: A list of sorted image filenames with '.png', '.jpg', or '.jpeg' extensions.
        """
        try:
            images = [f for f in os.listdir(self.folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
            images.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            return images
        except FileNotFoundError:
            print("Folder not found:", self.folder_path)
            return []

    def save_to_excel(self, image_variables):
        """
        Saves image classifications to an Excel file with '_labeled.xlsx' suffix.

        Parameters:
        - image_variables (dict): A dictionary with image names as keys and selected classifications as values.

        Output:
        - Excel file: The file is saved in the specified folder with '_labeled.xlsx' appended to the sample name.
        """
        # Collect classification labels for each image, removing file extensions
        image_labels = {os.path.splitext(img)[0]: var.get() for img, var in image_variables.items()}
        # Create a DataFrame from the dictionary and sort by 'Index'
        df = pd.DataFrame(list(image_labels.items()), columns=['Index', 'Class']).sort_values(by='Index')
        
        # Define output path and filename based on sample name
        output_file_name = f"{self.sample_name}_labeled.xlsx"
        output_path = os.path.join(self.folder_path, output_file_name)
        
        # Attempt to save DataFrame to Excel, with error handling
        try:
            df.to_excel(output_path, index=False)
            print("File saved:", output_path)
        except Exception as e:
            print("Error saving file:", e)

    def setup_gui(self):
        """
        Sets up and launches the GUI for image classification.

        Allows users to scroll through images, select classifications from a dropdown menu, and save the results.

        Input:
        - None (uses instance attributes for image list and folder path)
        
        Output:
        - Launches a Tkinter GUI for viewing images and selecting classifications.
        """
        root = Tk()
        root.geometry("800x600")

        canvas = Canvas(root)
        scrollbar = Scrollbar(root, command=canvas.yview)
        frame = Frame(canvas)
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.create_window((0, 0), window=frame, anchor="nw")

        image_variables = {}
        images_per_page = 100
        current_page = 0

        def update_scroll_region():
            frame.update_idletasks()
            canvas.config(scrollregion=canvas.bbox("all"))

        def load_page(page):
            # Clear current frame contents
            for widget in frame.winfo_children():
                widget.destroy()

            # Load images for the current page
            start = page * images_per_page
            end = start + images_per_page
            for i, image_name in enumerate(self.images[start:end], start=start):
                try:
                    img_path = os.path.join(self.folder_path, image_name)
                    img = Image.open(img_path).resize((250, 250), Image.Resampling.LANCZOS)  # Updated Resampling method
                    img = ImageTk.PhotoImage(img)

                    # Display image
                    label = Label(frame, image=img)
                    label.image = img
                    label.grid(row=i % images_per_page, column=0, padx=10, pady=10)

                    # Display image name without extension
                    image_name_without_extension = os.path.splitext(image_name)[0]
                    name_label = Label(frame, text=image_name_without_extension)
                    name_label.grid(row=i % images_per_page, column=1, padx=10, pady=10)

                    # Dropdown for classification options
                    var = StringVar(root)
                    var.set("Delete")  # Default selection
                    dropdown = OptionMenu(frame, var, *self.classification_options)
                    dropdown.grid(row=i % images_per_page, column=2, padx=10, pady=10)

                    image_variables[image_name] = var
                except Exception as e:
                    print(f"Error loading {image_name}: {e}")

            update_scroll_region()

        def change_page(delta):
            nonlocal current_page
            current_page = max(0, min(current_page + delta, len(self.images) // images_per_page))
            load_page(current_page)

        def on_mousewheel(event):
            canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

        # Navigation and Save buttons
        prev_button = Button(root, text="Previous", command=lambda: change_page(-1))
        prev_button.pack(side="left")
        next_button = Button(root, text="Next", command=lambda: change_page(1))
        next_button.pack(side="right")
        save_button = Button(root, text="Save", command=lambda: self.save_to_excel(image_variables))
        save_button.pack(side="bottom")

        load_page(current_page)
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        root.bind("<MouseWheel>", on_mousewheel)

        root.mainloop()

In [4]:
# folder_path = r'Seg_Images\USU-4183B 250-355 Elemental Map_aligned\Selected_and_Resized'
folder_path = r'Seg_Images\USU-4183B 150-250 Elemental Map\Selected_and_Resized'
sample_name = r'USU-4183B-GS-150-250'

- Q: quartz
- KSpar: K-feldspar
- Multiphase: either one mineral growing into one another or perthite or anti-perthite this kinda mixture
- PSpar: plagioclase feldspar
- M: Muscovite

In [5]:
classification_options = ["Q","KSpar","Multiphase","M","PSpar"]
labeler = ImageLabeler(folder_path=folder_path, classification_options=classification_options,sample_name=sample_name)
labeler.setup_gui()

File saved: Seg_Images\USU-4183B 150-250 Elemental Map\Selected_and_Resized\USU-4183B-GS-150-250_labeled.xlsx


# Rename the Images

You need to put images in a way fast.ai can load.

path/class1/image1.jpg

path/class1/image2.jpg

path/class2/image3.jpg

In [6]:
import os
import pandas as pd
from PIL import Image
from pathlib import Path

class ImageRenamer:
    """
    A class to rename and organize labeled images into folders for each label.

    Attributes:
    - to_be_renamed_folder (str): The path to the folder containing images and label Excel file.
    - to_be_trained_folder (str): The path to the target folder where renamed images will be saved.
    - sample_name (str): The name of the sample to include in the renamed files.

    Methods:
    - load_labels: Reads the Excel file to load image labels.
    - rename_and_save_images: Renames, organizes, and saves images into subfolders based on labels.
    """

    def __init__(self, to_be_renamed_folder, to_be_trained_folder, sample_name):
        """
        Initializes the ImageRenamer class with folder paths and sample name.

        Parameters:
        - to_be_renamed_folder (str): Path to the folder with images and label Excel file.
        - to_be_trained_folder (str): Path to the folder for saving renamed images.
        - sample_name (str): Name of the sample, included in renamed files.
        """
        self.to_be_renamed_folder = to_be_renamed_folder
        self.to_be_trained_folder = to_be_trained_folder
        self.sample_name = sample_name
        self.labels = self.load_labels()

    def load_labels(self):
        """
        Loads image labels from the Excel file in `to_be_renamed_folder`.

        Returns:
        - dict: A dictionary mapping image indices to labels.
        """
        try:
            # Locate the Excel file with '_labeled.xlsx' suffix in the directory
            excel_file = next((f for f in os.listdir(self.to_be_renamed_folder) if f.endswith('_labeled.xlsx')), None)
            if not excel_file:
                raise FileNotFoundError("No labeled Excel file found in the directory.")

            # Read the Excel file into a DataFrame
            df = pd.read_excel(os.path.join(self.to_be_renamed_folder, excel_file))
            # Convert DataFrame to dictionary with {Index: Label} mapping
            labels = dict(zip(df['Index'], df['Class']))
            return labels
        except Exception as e:
            print("Error loading labels:", e)
            return {}

    def rename_and_save_images(self):
        """
        Renames and saves images into subfolders based on labels. Skips images labeled 'Delete'.

        - For each unique label, creates a subfolder under `to_be_trained_folder` if it doesn't exist.
        - Images are renamed in the format: `Label_SampleName_Index`.
        """
        if not self.labels:
            print("No labels loaded, exiting.")
            return

        for index, label in self.labels.items():
            # Skip any image labeled as 'Delete'
            if label == "Delete":
                print(f"Image {index} labeled as 'Delete', skipping.")
                continue

            # Construct the original file name and the new name
            original_file_name = f"{index}.png"  # Assuming all images are .png format
            original_path = os.path.join(self.to_be_renamed_folder, original_file_name)
            if not os.path.exists(original_path):
                print(f"Image file {original_file_name} not found, skipping.")
                continue

            # Create the subfolder for the label if it doesn't exist
            label_folder = os.path.join(self.to_be_trained_folder, label)
            os.makedirs(label_folder, exist_ok=True)

            # Construct the new filename
            new_name = f"{label}_{self.sample_name}_{index}.png"
            new_path = os.path.join(label_folder, new_name)

            # Open, rename, and save the image to the appropriate subfolder
            try:
                img = Image.open(original_path)
                img.save(new_path)
                print(f"Renamed and saved image as: {new_path}")
            except Exception as e:
                print(f"Error saving {original_file_name}: {e}")

In [7]:

# to_be_renamed_folder = r"Seg_Images\USU-4183B 250-355 Elemental Map_aligned\Selected_and_Resized"
# to_be_renamed_folder = r'Seg_Images\USU-4162A 250-355 Elemental Map_aligned\Selected_and_Resized'
to_be_renamed_folder = r"Seg_Images\USU-4183B 150-250 Elemental Map\Selected_and_Resized"
to_be_trained_folder = "3_To_be_Trained"

renamer = ImageRenamer(to_be_renamed_folder, to_be_trained_folder, sample_name)
renamer.rename_and_save_images()


Renamed and saved image as: 3_To_be_Trained\KSpar\KSpar_USU-4183B-GS-150-250_1.png
Renamed and saved image as: 3_To_be_Trained\KSpar\KSpar_USU-4183B-GS-150-250_10.png
Renamed and saved image as: 3_To_be_Trained\KSpar\KSpar_USU-4183B-GS-150-250_11.png
Renamed and saved image as: 3_To_be_Trained\KSpar\KSpar_USU-4183B-GS-150-250_12.png
Renamed and saved image as: 3_To_be_Trained\KSpar\KSpar_USU-4183B-GS-150-250_13.png
Renamed and saved image as: 3_To_be_Trained\Multiphase\Multiphase_USU-4183B-GS-150-250_14.png
Renamed and saved image as: 3_To_be_Trained\KSpar\KSpar_USU-4183B-GS-150-250_15.png
Renamed and saved image as: 3_To_be_Trained\Multiphase\Multiphase_USU-4183B-GS-150-250_16.png
Renamed and saved image as: 3_To_be_Trained\KSpar\KSpar_USU-4183B-GS-150-250_17.png
Renamed and saved image as: 3_To_be_Trained\KSpar\KSpar_USU-4183B-GS-150-250_18.png
Renamed and saved image as: 3_To_be_Trained\Multiphase\Multiphase_USU-4183B-GS-150-250_19.png
Renamed and saved image as: 3_To_be_Trained\KSp